# Sort catchment shape
Sorts the catchment shape by `gruId` first and `hruId` second, to ensure HRU order follows SUMMA conventions. Not strictly necessary for cases where each GRU contains one HRU, but essential for cases where the GRUs contain multiple HRUs.

In [1]:
# modules
import geopandas as gpd
from pathlib import Path
from shutil import copyfile
from datetime import datetime

#### Control file handling

In [8]:
# Easy access to control file folder
controlFolder = Path('../0_control_files')

In [4]:
# Store the name of the 'active' file in a variable
controlFile = 'control_active.txt'

In [5]:
# Function to extract a given setting from the control file
def read_from_control( file, setting ):
    
    # Open 'control_active.txt' and ...
    with open(file) as contents:
        for line in contents:
            
            # ... find the line with the requested setting
            if setting in line:
                break
    
    # Extract the setting's value
    substring = line.split('|',1)[1]      # Remove the setting's name (split into 2 based on '|', keep only 2nd part)
    substring = substring.split('#',1)[0] # Remove comments, does nothing if no '#' is found
    substring = substring.strip()         # Remove leading and trailing whitespace, tabs, newlines
       
    # Return this value    
    return substring

In [6]:
# Function to specify a default path
def make_default_path(suffix):
    
    # Get the root path
    rootPath = Path( read_from_control(controlFolder/controlFile,'root_path') )
    
    # Get the domain folder
    domainName = read_from_control(controlFolder/controlFile,'domain_name')
    domainFolder = 'domain_' + domainName
    
    # Specify the forcing path
    defaultPath = rootPath / domainFolder / suffix
    
    return defaultPath

#### Find location of catchment shapefile

In [14]:
# Catchment shapefile path & name
catchment_path = read_from_control(controlFolder/controlFile,'catchment_shp_path')
catchment_name = read_from_control(controlFolder/controlFile,'catchment_shp_name')

In [15]:
# Specify default path if needed
if catchment_path == 'default':
    catchment_path = make_default_path('shapefiles/catchment') # outputs a Path()
else:
    catchment_path = Path(catchment_path) # make sure a user-specified path is a Path()

In [16]:
# Find GRU and HRU variables
gru_id = read_from_control(controlFolder/controlFile,'catchment_shp_gruid')
hru_id = read_from_control(controlFolder/controlFile,'catchment_shp_hruid')

#### Open shape and sort

In [21]:
# Open the shape
shp = gpd.read_file(catchment_path/catchment_name)

In [23]:
# Sort
shp = shp.sort_values(by=[gru_id,hru_id])

In [24]:
# Save
shp.to_file(catchment_path/catchment_name)

#### Code provenance
Generates a basic log file in the domain folder and copies the control file and itself there.

In [25]:
# Set the log path and file name
logPath = catchment_path
log_suffix = '_sort_shape.txt'

In [26]:
# Create a log folder
logFolder = '_workflow_log'
Path( logPath / logFolder ).mkdir(parents=True, exist_ok=True)

In [28]:
# Copy this script
thisFile = '1_sort_catchment_shape.ipynb'
copyfile(thisFile, logPath / logFolder / thisFile);

In [27]:
# Get current date and time
now = datetime.now()

In [29]:
# Create a log file 
logFile = now.strftime('%Y%m%d') + log_suffix
with open( logPath / logFolder / logFile, 'w') as file:
    
    lines = ['Log generated by ' + thisFile + ' on ' + now.strftime('%Y/%m/%d %H:%M:%S') + '\n',
             'Sorted catchment shape by GRU ID first and HRU ID second to follow SUMMA conventions.']
    for txt in lines:
        file.write(txt)  